# Task: Data Extraction and VectorDB Creation

In [2]:
import fitz  # PyMuPDF
import re
import os

c:\Users\MARTIALEAGLE\AppData\Local\Programs\Python\Python311\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [14]:
from langchain_community.document_loaders import DirectoryLoader

# Load documents from a directory
directory_path = './personalDocuments'
loader = DirectoryLoader(directory_path, glob="**/*")
docs = loader.load()
len(docs)

8

In [16]:
docs

[Document(page_content='ASHWANTHRAM K L\n\n(cid:131) 8681849991 # k.l.ashwanthram@gmail.com (cid:239) ashwanthram § AshwanthramKL\n\nEDUCATION\n\nCoimbatore Institute of Technology, Coimbatore Bachelor of Engineering (B.E) Computer Science and Engineeering TECHNICAL SKILLS\n\n07/2020 – 05/2024\n\nProgramming Languages: Python, Javascript, C++ Frameworks & Libraries: Streamlit, Gradio, Langchain, Autogen (Multiagent), OpenAI Platform, together.ai, HuggingFace Prompt Engineering Techniques: Reﬂexion, Chain-of-thought, Tree-of-thought, In-Context Learning, ReAct, Expert Persona Prompting, Meta Prompting, Multi-Agent Debate Prompting Machine Learning: Large Language Models (LLMs), Retrieval Augmented Generation (RAG), LLM Finetuning, Information Retrieval, Transformers, Word2Vec, Node2Vec, NumPy, Scikit-Learn, Pandas, NLTK, Matplotlib, Tensorﬂow Software & Tools: FAISS, ChromaDB, LlamaIndex, Pinecone, MySQL, Git, Github, PromptLayer, AWS SageMaker, Google Colab, VS Code, Jupyter Notebooks,

In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize the RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=512,  # Since max input seq of embeddings model is 512 
    chunk_overlap=100,  # 1: 5 ratio
    length_function=len,
    is_separator_regex=False,
)

# Split the documents into chunks
text_chunks = text_splitter.split_documents(docs)

In [22]:
from langchain_community.vectorstores import FAISS
from InstructorEmbedding import INSTRUCTOR
pip install -U langchain-community
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

# Initialise the embedding model
embedding_model = HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-xl')

# Build FAISS vector store
vector_store = FAISS.from_documents(text_chunks, embedding_model, normalize_L2=True)
vector_store.save_local('vectorstore')

load INSTRUCTOR_Transformer


c:\Users\MARTIALEAGLE\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


max_seq_length  512


# Task: Query Formulation and Retrieval

In [41]:
from langchain.vectorstores import FAISS
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

import streamlit as st
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

In [25]:
# Load the vectordb from local storage
new_db = FAISS.load_local("vectorstore", HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-xl'))

load INSTRUCTOR_Transformer
max_seq_length  512


In [53]:
# Retriever
retriever = new_db.as_retriever(search_kwargs={"k": 7, "hnsw:space": "cosine"})


In [40]:
from dotenv import load_dotenv

# Set api key
load_dotenv("D:\FarmwiseAI\Reddit\.env")

# Load the API key from the environment
api_key = os.environ.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

# Language Model
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0, model_kwargs={"top_p": 0.5})

In [43]:
condense_q_system_prompt = """Given a chat history and the latest user question \
which might reference the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
condense_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
condense_q_chain = condense_q_prompt | llm | StrOutputParser()

In [44]:
qa_system_prompt = """You are an AI assistant designed to provide comprehensive information about Ashwanthram. Your primary role is to help users understand various aspects of Ashwanthram's life, work, interests, achievements, etc. based on the context provided. Below is a detailed guide on how to interact and provide valuable assistance:

User Profile:
Name: Ashwanthram
Profession: GenAI Data Scientist
Experience: 10 months working with LLMs and RAG to build applications such as an LLM-powered social media recommendation system and a personalized travel planner application.
Interests: Tech, generative AI, health, fitness, longevity, self-help podcasts, and working out.

Interaction Guidelines:
* Be Clear and Informative: Provide detailed and accurate information about Ashwanthram. Avoid overly technical jargon unless necessary.
* Be Supportive and Encouraging: Encourage users to learn more about Ashwanthram's interests and achievements. Offer positive reinforcement.
* Be Detailed and Comprehensive: Provide in-depth information about Ashwanthram's work, interests, and achievements. Include relevant examples and details.

Example Dialogue:
User: Can you tell me about Ashwanthram's professional background?

AI Assistant: Ashwanthram is a GenAI Data Scientist with 9 months of experience working with Large Language Models (LLMs) and Retrieval-Augmented Generation (RAG). He has developed applications such as an LLM-powered social media recommendation system and a personalized travel planner application. He is passionate about continuing to leverage LLMs and work on cutting-edge technologies.

Context:
{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def format_docs(documents):
    return "\n\n".join(document.page_content for document in documents)

def condense_question(input: dict):
    if input.get("chat_history"):
        return condense_q_chain
    else:
        return input["question"]

rag_chain = (
    RunnablePassthrough.assign(context=condense_question | retriever | format_docs)
    | qa_prompt
    | llm
)

In [49]:
# Initialize chat history
chat_history = []

# Streamlit app setup
# Set page configuration with title and layout
st.set_page_config(page_title='Ashwanth\'s Personal AI Assistant', layout='wide')

st.title("Ashwanth's Personal AI Assistant")

# Initialize session state for chat history if not already present
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []

# Function to handle message sending
def send_message():
    user_input = st.session_state.user_input
    if user_input:
        # Process user message
        ai_msg = rag_chain.invoke({"question": user_input, "chat_history": st.session_state['chat_history']})
        st.session_state['chat_history'].extend([HumanMessage(content=user_input), ai_msg])
        # Print chat history for debugging
        print("Updated chat history:", st.session_state['chat_history'])


        # Clear the input box
        st.session_state.user_input = ""

# Use columns for better layout
col1, col2 = st.columns([3, 1])

# Container for chat history
# Initialize session state for chat history if not already present
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []

# Container for user input
with col2:
    with st.container():
        # Text input for user message
        # user_input = st.text_input("Your message:", key="user_input", on_change=send_message, args=())
        user_input = st.text_area("Your message:", key="user_input", on_change=send_message, args=(), height=500)
        
        # Send button
        send_btn = st.button("Send", on_click=send_message)
        
# CSS for making the first column scrollable
css='''
<style>
    section.main>div {
        padding-bottom: 1rem;
    }
    [data-testid="column"]>div>div>div>div>div {
        overflow: auto;
        height: 80vh;
    }
</style>
'''

st.markdown(css, unsafe_allow_html=True)

# JavaScript to manage the scroll position
scroll_js = """
<script>
function maintainScrollPosition(){
    var container = document.querySelector("[data-testid='column']>div>div>div>div>div");
    var scrollPosition = localStorage.getItem('scrollPosition');
    if (scrollPosition) {
        container.scrollTop = scrollPosition;
    }
    container.addEventListener('scroll', function() {
        localStorage.setItem('scrollPosition', container.scrollTop);
    });
}
// Call the function when the page loads
setTimeout(maintainScrollPosition, 500);
</script>
"""

st.markdown(scroll_js, unsafe_allow_html=True)


DeltaGenerator()

In [56]:
!streamlit run streamlit_app.py